This tutorial is adapted from https://towardsdatascience.com/nlp-extracting-the-main-topics-from-your-dataset-using-lda-in-minutes-21486f5aa925

## Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is the 20newsgroup dataset that is available from sklearn. This dataset has news articles grouped into 20 news categories

http://qwone.com/~jason/20Newsgroups/

https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups 

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', shuffle = True)
newsgroups_test = fetch_20newsgroups(subset='test', shuffle = True)

In [ ]:
print("\n".join(list(newsgroups_train.target_names)))

As you can see that there are some distinct themes in the news categories like sports, religion, science, technology, politics etc.

In [ ]:
# Lets look at some sample news
print(newsgroups_train.data[0])

In [ ]:
print(newsgroups_train.target.shape)

## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **stemmed** - words are reduced to their root form.


In [ ]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim OR conda install -c anaconda gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [ ]:
words = ['caresses', 'flies', 'dies', 'mules', 'denied',
          'died', 'agreed', 'owned', 'humbled', 'sized',
          'meeting', 'stating', 'siezing', 'itemization',
          'sensational', 'traditional', 'reference', 'colonizer',
          'plotted']

stemmer = PorterStemmer()
stems = [stemmer.stem(w) for w in words]
print("\t".join(stems))

In [ ]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
# Tokenize, stem and removing stopwords
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(stemmer.stem(token))
    return result

In [ ]:
'''
Preview a document after preprocessing
'''
doc_sample = 'This disk has failed many times. I would like to get it replaced.'

print("Tokenized and stemmed document: ")
print(preprocess(doc_sample))

In [ ]:
print(gensim.parsing.preprocessing.STOPWORDS)

Let's now preprocess all the news headlines we have. To do that, we iterate over the list of documents in our training sample

In [ ]:
processed_docs = []
for doc in newsgroups_train.data:
    processed_docs.append(preprocess(doc))

In [ ]:
'''
Preview 'processed_docs'
'''
print(" ".join(processed_docs[2]))

## Step 3: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [ ]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''


In [ ]:
'''
Checking dictionary created
'''
list(dictionary.items())[:20]

### Gensim filter_extremes

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [ ]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 50% of all documents
'''


### Gensim doc2bow

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [ ]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''


In [ ]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("word", bow_doc_x[i][0], dictionary[bow_doc_x[i][0]], bow_doc_x[i][1])

## Step 4: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [ ]:
# LDA mono-core -- fallback code in case LdaMulticore throws an error on your machine
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 50)

# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''


In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \n Word: {}".format(idx, topic))
    print("\n")

# Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: Graphics Cards
* 1: Religion
* 2: Space
* 3: Politics
* 4: Gun Violence
* 5: Technology
* 6: Sports
* 7: Encryption 

In [ ]:
# conda install -c conda-forge pyldavis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
for_viz = gensimvis.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(for_viz)

## Step 6: Testing model on unseen document ##

In [ ]:
num = 100
unseen_document = newsgroups_test.data[num]
print(unseen_document)

In [ ]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

#lda_model[bow_vector]

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t \t Topic: {}".format(score, lda_model.print_topic(index, 5)))

The model correctly classifies the unseen document with 'x'% probability to the X category.